### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [213]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

# school data clone
school_data_clone = school_data



## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [214]:
#create bins for a Pass Fail Classification based on a passing Rate fo 70%
bins = [0, 69, 100]
group_names = ["Fail","Pass"]

school_data_complete["Pass/Fail (Math)"] = pd.cut(school_data_complete["math_score"], bins, labels=group_names)
school_data_complete["Pass/Fail (Reading)"] = pd.cut(school_data_complete["reading_score"], bins, labels=group_names)

In [215]:
#get the school_type
school_type = school_data[["school_name", "type"]]

In [216]:
#calculate the number of schools in the data set 
school_count = len(school_data_complete["school_name"].unique())

In [217]:
#calculate the number of students in the data set 
student_count = len(school_data_complete["Student ID"].unique())

In [218]:
# group data 
grouped_school_data = school_data_complete.groupby(["school_name", "type"])

In [219]:
#calculate the overall sum of the budget from the data set 
school_budget = sum(grouped_school_data["budget"].mean())

In [220]:
#calculate the average math score
avg_math_score = school_data_complete["math_score"].mean()

In [221]:
#calculate the average reading score
avg_reading_score = school_data_complete["reading_score"].mean()

In [222]:
# calculate passing rate (overall_avergae)
passing_rate = (avg_math_score + avg_reading_score) / 2

In [223]:
#Calculate cuts based on Math and reading scores
math_pass_fail = pd.DataFrame(pd.cut(school_data_complete["math_score"], bins, labels=group_names))
reading_pass_fail = pd.DataFrame(pd.cut(school_data_complete["reading_score"], bins, labels=group_names))

In [224]:
# find the count for students Passing Math
students_who_passed_math = math_pass_fail[math_pass_fail["math_score"]=="Pass"].count()

In [225]:
# find the count for students Passing Reading
students_who_passed_reading = reading_pass_fail[reading_pass_fail["reading_score"]=="Pass"].count()

In [226]:
pct_passing_math = (students_who_passed_math["math_score"] /student_count) * 100

In [227]:
pct_passing_reading = (students_who_passed_reading["reading_score"] /student_count)  * 100

In [228]:
#Create Dataframe for Summary Stats
district_summary_df = pd.DataFrame({"Total Schools" : [school_count] ,
                                    "Total Students" : [student_count],
                                    "Total Budget" : [school_budget],
                                    "Average Math Score" : [avg_math_score],
                                    "Average Reading Score" : [avg_reading_score],
                                    "% Passing Math" : [pct_passing_math],
                                    "% Passing Reading" : [pct_passing_reading],
                                    "% Overall Passing Rate" : [passing_rate] })


In [229]:
#apply formatting
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:.2f}".format)
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)

In [230]:
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170",$24649428.00,78.985371,81.87784,74.980853,85.805463,80.431606


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [231]:
school_mean_data = grouped_school_data["budget"].mean()

In [232]:
school_student_count = grouped_school_data["Student ID"].count()

In [233]:
school_budget_per_student = pd.DataFrame(school_mean_data / school_student_count)

In [234]:
school_avg_math_score_data = grouped_school_data["math_score"].mean()

In [235]:
school_avg_reading_score_data = grouped_school_data["reading_score"].mean()

In [236]:
#perform merges
combined_mean_student_count = pd.merge(school_mean_data, school_student_count, on="school_name", how="inner")
combined_avg_math_and_reading = pd.merge(school_avg_math_score_data, school_avg_reading_score_data, on="school_name", how="inner")

In [237]:
school_results_raw = pd.merge(combined_mean_student_count, combined_avg_math_and_reading, on="school_name", how="inner")
school_results = pd.merge(school_budget_per_student, school_results_raw, on="school_name", how="inner")
school_results_renamed = school_results.rename(columns={
                                "Student ID": "Total Students",
                                "budget": "Total School Budget",
                                "math_score": "Average Math Score",
                                "reading_score": "Average Reading Score"
                                })

In [238]:
total_passing_math_by_school = school_data_complete.loc[school_data_complete["Pass/Fail (Math)"]=="Pass",:]
math_passes_grouped_by_school = total_passing_math_by_school.groupby(["School ID", "school_name", "type"])
math_passes_by_school = pd.DataFrame((math_passes_grouped_by_school["Pass/Fail (Math)"].count() / grouped_school_data["Student ID"].count()) * 100)


In [239]:
total_passing_reading_by_school = school_data_complete.loc[school_data_complete["Pass/Fail (Reading)"]=="Pass",:]
reading_passes_grouped_by_school = total_passing_reading_by_school.groupby(["School ID", "school_name", "type"])
reading_passes_by_school = pd.DataFrame((reading_passes_grouped_by_school["Pass/Fail (Reading)"].count() / grouped_school_data["Student ID"].count()) * 100)

In [240]:
math_reading_results_by_school_raw = pd.merge(math_passes_by_school, reading_passes_by_school, on ="school_name", how="inner"  )
math_reading_results = math_reading_results_by_school_raw.rename(columns={"0_x": "% Passing Math", "0_y": "% Passing Reading"})
math_reading_results["Overall Pass Rate"] = (math_reading_results["% Passing Math"] + math_reading_results["% Passing Reading"]) / 2

In [241]:
schools_raw = pd.merge(school_results_renamed,math_reading_results, on="school_name")
school_performance_raw = pd.merge(school_data, schools_raw, on="school_name", how="inner")

In [242]:
#clean-up column names
school_performance_rename = school_performance_raw.rename(columns={"type": "School Type", 0: "Per Student Budget", "Overall Pass Rate" : "Overall Passing Rate"})


In [243]:
#apply filters to get a reduce set of columns
school_performance_filtered = school_performance_rename.filter(["school_name", "Total Students", "School Type","Total School Budget", "Per Student Budget", "Average Math Score" \
                                                                "Average Reading Score", "% Passing Math", "% Passing Reading", "Overall Passing Rate"])


In [244]:
#apply formatting wiith the map function
school_performance_filtered["Total School Budget"] = school_performance_filtered["Total School Budget"].map("${:.2f}".format)
school_performance_filtered["Per Student Budget"] = school_performance_filtered["Per Student Budget"].map("${:.2f}".format)
school_performance_filtered["Total Students"] = school_performance_filtered["Total Students"].map("{:,}".format)


In [245]:
#define school_name column as index
school_performance_indexed = school_performance_filtered.set_index("school_name")

In [246]:
# sort data descending to get the top 5 schools by overall performance
school_performance_indexed.sort_values(by="Overall Passing Rate", ascending = False).head(5)

,Total Students,School Type,Total School Budget,Per Student Budget,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,,,,,,,
Cabrera High School,"1,858",Charter,$1081356.00,$582.00,94.133477,97.039828,95.586652
Thomas High School,"1,635",Charter,$1043130.00,$638.00,93.272171,97.308869,95.290520
Pena High School,962,Charter,$585858.00,$609.00,94.594595,95.945946,95.270270
Griffin High School,"1,468",Charter,$917500.00,$625.00,93.392371,97.138965,95.265668
Wilson High School,"2,283",Charter,$1319574.00,$578.00,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [247]:
# sort data ascending to get the bottom 5 schools by overall performance
school_performance_indexed.sort_values(by="Overall Passing Rate", ascending = True).head(5)

,Total Students,School Type,Total School Budget,Per Student Budget,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,,,,,,,
Rodriguez High School,"3,999",District,$2547363.00,$637.00,66.366592,80.220055,73.293323
Figueroa High School,"2,949",District,$1884411.00,$639.00,65.988471,80.739234,73.363852
Huang High School,"2,917",District,$1910635.00,$655.00,65.683922,81.316421,73.500171
Johnson High School,"4,761",District,$3094650.00,$650.00,66.057551,81.222432,73.639992
Ford High School,"2,739",District,$1763916.00,$644.00,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [ ]:
#Math Score by Grade
school_data_formatted = school_data_complete.replace("9th", "09th")
math_score_by_grade = pd.crosstab(school_data_formatted["school_name"], school_data_formatted["grade"], values=school_data_formatted["math_score"], aggfunc='mean')
math_score_by_grade.columns.names = [None]
math_score_by_grade

## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [ ]:
#Reading Score by Grade
#school_data_formatted = school_data_complete.replace("9th", "09th")
reading_score_by_grade = pd.crosstab(school_data_formatted["school_name"], school_data_formatted["grade"], values=school_data_formatted["reading_score"], aggfunc='mean')
reading_score_by_grade.columns.names = [None]
reading_score_by_grade

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [313]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [336]:
#get dataset from the per school analysis above
school_dataset = school_results_renamed.rename(columns={0: "Per Student Spend"})

#apply the binning based on the Spending bin cuts
school_dataset = pd.DataFrame(pd.cut(school_dataset["Per Student Spend"], spending_bins, labels=group_names))


school_budget_and_results_raw = pd.DataFrame(school_results)
school_budget_and_results_raw["average_spending"] = school_budget_and_results_raw["budget"] / school_budget_and_results_raw["Student ID"]
school_budget_and_spending_raw = pd.DataFrame(pd.cut(school_budget_and_results_raw["average_spending"], spending_bins, labels=group_names))

school_spending_group = school_budget_and_spending_raw.groupby("average_spending")

#print(school_spending_group["Average Math Score"].mean())


#school_spending_vs_results.sort_values(by="Overall Pass Rate", ascending = False).head(5)

In [286]:

school_spending_vs_results = pd.merge(math_reading_results, school_budget_and_spending_raw, on="school_name", how="inner")


In [290]:

school_spending_vs_results_group = school_spending_vs_results.groupby("average_spending")


In [291]:
pct_passing_math_by_spending = school_spending_vs_results_group["% Passing Math"].mean()
pct_passing_reading_by_spending = school_spending_vs_results_group["% Passing Reading"].mean()

## Scores by School Size

* Perform the same operations as above, based on school size.

In [34]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

## Scores by School Type

* Perform the same operations as above, based on school type.

In [295]:
# Get Average Math and Reading Scores grouped by School Type
school_type_groupby_data = school_data_complete.groupby(["type"])

avg_math_score_by_type = pd.DataFrame(school_type_groupby_data["math_score"].mean())
avg_reading_score_by_type = pd.DataFrame(school_type_groupby_data["reading_score"].mean())
avg_math_reading_score_by_type = pd.merge(avg_math_score_by_type, avg_reading_score_by_type, on="type")


In [296]:
# Rename Columns to be cleaner / more report friendly
school_data_complete_rename = pd.DataFrame(school_data_complete.rename(columns={"Pass/Fail (Math)": "Math_Results", "Pass/Fail (Reading)": "Reading_Results"}))


In [297]:
#Get a count of the Number of Students grouped by district and Math
school_data_complete_rename_math_grouby_data = school_data_complete_rename.groupby(["type"])
total_math_students_by_type_raw = pd.DataFrame(school_data_complete_rename_math_grouby_data["Math_Results"].count())
total_math_students_by_type = total_math_students_by_type_raw.rename(columns={"Math_Results": "total_math_students"})


In [298]:
#Get a count of the Number of Students grouped by district and Reading
school_data_complete_rename_reading_grouby_data = school_data_complete_rename.groupby(["type"])
total_reading_students_by_type_raw = pd.DataFrame(school_data_complete_rename_reading_grouby_data["Reading_Results"].count())
total_reading_students_by_type = total_reading_students_by_type_raw.rename(columns={"Reading_Results": "total_reading_students"})


In [299]:
# Get All math passes
all_math_passes = pd.DataFrame(school_data_complete_rename[school_data_complete_rename.Math_Results == 'Pass'])
all_math_passes_filtered_columns = all_math_passes[["type", "size", "Math_Results"]]
type_math_pass_groupby_data = all_math_passes_filtered_columns.groupby(["type"])
type_math_pass_raw = pd.DataFrame(type_math_pass_groupby_data["Math_Results"].count())
type_math_passes = type_math_pass_raw.rename(columns={"Math_Results": "total_math_passes"})

In [300]:
# Get All reading passes
all_reading_passes = pd.DataFrame(school_data_complete_rename[school_data_complete_rename.Reading_Results == 'Pass'])
all_reading_passes_filtered_columns = all_reading_passes[["type", "size", "Reading_Results"]]
type_reading_pass_groupby_data = all_reading_passes_filtered_columns.groupby(["type"])
type_reading_pass_raw = pd.DataFrame(type_reading_pass_groupby_data["Reading_Results"].count())
type_reading_passes = type_reading_pass_raw.rename(columns={"Reading_Results": "total_reading_passes"})

In [301]:
# Perform Merges
student_totals_by_subject_and_type = pd.merge(total_math_students_by_type, total_reading_students_by_type, on="type")
student_passes_by_subject_and_type = pd.merge(type_math_passes, type_reading_passes, on="type")
student_totals_by_type = pd.merge(student_passes_by_subject_and_type, student_totals_by_subject_and_type, on="type")
type_results_for_math_and_reading_raw = pd.merge(avg_math_reading_score_by_type, student_totals_by_type, on="type" )

In [302]:
# Compute Percentage Passes for Math and Reading by Type
type_results_for_math_and_reading_raw["% Passing Math"] = (type_results_for_math_and_reading_raw["total_math_passes"] / \
                                                          type_results_for_math_and_reading_raw["total_math_students"]) * 100

type_results_for_math_and_reading_raw["% Passing Reading"] = (type_results_for_math_and_reading_raw["total_reading_passes"] / \
                                                          type_results_for_math_and_reading_raw["total_reading_students"]) * 100

type_results_for_math_and_reading_raw["% Overall Passing Rate"] = ((type_results_for_math_and_reading_raw["total_math_passes"] + \
                                                          type_results_for_math_and_reading_raw["total_reading_passes"]) / \
                                                         (type_results_for_math_and_reading_raw["total_math_students"] + \
                                                          type_results_for_math_and_reading_raw["total_reading_students"] ) * 100)


In [305]:
type_results_for_math_and_reading_filtered = type_results_for_math_and_reading_raw.filter(["type", "math_score", "reading_score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"])
type_results_for_math_and_reading = type_results_for_math_and_reading_filtered.rename(columns={"type": "School Type", "math_score": "Average Math Score", "reading_score": "Average Reading Score"})


In [304]:
type_results_for_math_and_reading

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
type,,,,,
Charter,83.406183,83.902821,93.701821,96.645891,95.173856
District,76.987026,80.962485,66.518387,80.905249,73.711818
